In [37]:
import os
os.environ["LLM_API_KEY"] = "sk-KNo006G2a48UVE3IxFlQEQ"
os.environ["LLM_BASE_URL"] = "https://api.gpt.mws.ru"

In [8]:
from agents import ActionSuggestionAgent

In [9]:
agent = ActionSuggestionAgent(verbose=True)
agent.invoke('Привет, преобразуй этот текст в строчные БУКВЫ!!!')

c:\Users\Полина\MWS-GPT-Contact-Center-Assistant\app\backend\agents\ActionSuggestionAgent.py:13: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(




> Entering new AgentExecutor chain...
Используя доступный инструмент lower, который преобразует текст в строчные буквы, я могу ответить на вопрос.
Action: lower
Action Input: БУКВЫ!!!
Observation: буквы!!!
Thought:I now know the final answer
Final Answer: буквы!!!

> Finished chain.


{'input': 'Привет, преобразуй этот текст в строчные БУКВЫ!!!',
 'output': 'буквы!!!'}

In [17]:
import os
from abc import ABC


class BaseAgent(ABC):
    LLM_API_KEY: str = os.getenv("LLM_API_KEY")
    LLM_BASE_URL: str = os.getenv("LLM_BASE_URL")

    def __new__(cls, verbose: bool = False): ...

    def _llm(verbose: bool): ...

    def _tools(): ...


In [26]:
import os
from langchain_core.runnables import Runnable
from langchain.prompts import ChatPromptTemplate
from langchain.output_parsers import OutputParser  # Пробуем использовать базовый OutputParser
from langchain_openai import ChatOpenAI

class EmotionAgent(Runnable):
    def __init__(self, verbose: bool = False):
        self.llm = ChatOpenAI(
            base_url=os.getenv("LLM_BASE_URL"),
            api_key=os.getenv("LLM_API_KEY"),
            model=os.getenv("LLM_EMOTION_AGENT", "gpt-3.5-turbo"),
            temperature=0,
            verbose=verbose,
            system_message="Ты — классификатор эмоций. Определи одну из: гнев, радость, нейтральное, тоска, удивление. Отвечай только одним словом.",
        )

        self.prompt = ChatPromptTemplate.from_messages([
            ("system", "Ты — классификатор эмоций. Определи одну из: гнев, радость, нейтральное, тоска, удивление. Отвечай только одним словом."),
            ("human", "{text}")
        ])

        # Используем OutputParser вместо JsonOutputParser или StrOutputParser
        self.chain = self.prompt | self.llm | OutputParser()

    def invoke(self, input_text: str) -> str:
        return self.chain.invoke({"text": input_text})


ImportError: cannot import name 'OutputParser' from 'langchain.output_parsers' (c:\ProgramData\anaconda3\Lib\site-packages\langchain\output_parsers\__init__.py)

In [28]:
class EmotionAgent(Runnable):
    def __init__(self, verbose: bool = False):
        self.llm = ChatOpenAI(
            base_url=os.getenv("LLM_BASE_URL"),
            api_key=os.getenv("LLM_API_KEY"),
            model=os.getenv("LLM_EMOTION_AGENT", "gpt-3.5-turbo"),
            temperature=0,
            verbose=verbose,
            system_message="Ты — классификатор эмоций. Определи одну из: гнев, радость, нейтральное, тоска, удивление. Отвечай только одним словом.",
        )

        self.prompt = ChatPromptTemplate.from_messages([
            ("system", "Ты — классификатор эмоций. Определи одну из: гнев, радость, нейтральное, тоска, удивление. Отвечай только одним словом."),
            ("human", "{text}")
        ])

        self.chain = self.prompt | self.llm  # Без парсера

    def invoke(self, input_text: str) -> str:
        response = self.chain.invoke({"text": input_text})
        return response['text']  # Возвращаем текстовый ответ


In [34]:
import os
from langchain_core.runnables import Runnable
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

class EmotionAgent(Runnable):
    def __init__(self, verbose: bool = False):
        self.llm = ChatOpenAI(
            base_url=os.getenv("LLM_BASE_URL"),
            api_key=os.getenv("LLM_API_KEY"),
            temperature=0,
            verbose=verbose,
            system_message="Ты — классификатор эмоций. Определи одну из: гнев, радость, нейтральное, тоска, удивление. Отвечай только одним словом."
        )

        self.prompt = ChatPromptTemplate.from_messages([
            ("system", "Ты — классификатор эмоций. Определи одну из: гнев, радость, нейтральное, тоска, удивление. Отвечай только одним словом."),
            ("human", "{text}")
        ])

        # Здесь удаляем ненужные параметры и просто используем prompt и llm
        self.chain = self.prompt | self.llm

    def invoke(self, input_text: str) -> str:
        response = self.chain.invoke({"text": input_text})
        return response['text']  # Возвращаем текстовый ответ


In [42]:
import os

from langchain.agents import AgentExecutor, AgentType, Tool, initialize_agent
from langchain_openai import ChatOpenAI

from BaseAgent import BaseAgent

ModuleNotFoundError: No module named 'BaseAgent'

In [43]:
import os
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.runnables import Runnable

class EmotionAgent(Runnable):
    def __new__(cls, verbose: bool = False):
        llm = cls._llm(verbose)
        tools = cls._tools()
        agent = initialize_agent(
            tools,
            llm,
            agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
            verbose=verbose,
        )
        return agent

    def _llm(verbose: bool):
        return ChatOpenAI(
            base_url=BaseAgent.LLM_BASE_URL,
            api_key=BaseAgent.LLM_API_KEY,
            model=os.getenv("LLM_ACTIONSUGGESTION_AGENT", "qwen2.5-32b-instruct"),
            verbose=verbose,
        )

    def _tools():
        def lower_tool(query: str):
            return query.lower()

        tools = [
            Tool(
                name="lower",
                func=lower_tool,
                description="Возвращает текст строчными буквами",
            ),
        ]

        return tools
emotion_agent = EmotionAgent(verbose=True)
emotion = emotion_agent.invoke("Мне так грустно, я просто не знаю, что делать...")
print("Эмоция:", emotion)



> Entering new AgentExecutor chain...
用户提供的是俄语文本，表达了他们感到非常沮丧，并不知道应该怎么做。虽然我有一个工具可以将文本转为小写，但在这个情境下，这个工具并不适用。我的目标是提供一些安慰和建议。

Thought: 我需要提供安慰，并建议应对沮丧的方法。
Final Answer: Я понимаю, что сейчас тебе трудно, и это нормально. Попробуй поговорить о своих чувствах с близкими тебе людьми, они могут поддержать и помочь найти решение. Также попробуй найти что-то, что приносит радость, например, заняться своим хобби или прогуляться на свежем воздухе. Помните, что чувства приходят и уходят, и ситуацию можно изменить.

> Finished chain.
Эмоция: {'input': 'Мне так грустно, я просто не знаю, что делать...', 'output': 'Я понимаю, что сейчас тебе трудно, и это нормально. Попробуй поговорить о своих чувствах с близкими тебе людьми, они могут поддержать и помочь найти решение. Также попробуй найти что-то, что приносит радость, например, заняться своим хобби или прогуляться на свежем воздухе. Помните, что чувства приходят и уходят, и ситуацию можно изменить.'}


In [ ]:
import os
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.runnables import Runnable
from langchain.agents import AgentExecutor, Tool, AgentType, initialize_agent

class EmotionAgent(Runnable):
    def __new__(cls, verbose: bool = False):
        llm = cls._llm(verbose)
        tools = cls._tools()
        agent = initialize_agent(
            tools,
            llm,
            agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
            verbose=verbose,
        )
        return agent

    def _llm(verbose: bool):
        return ChatOpenAI(
            base_url=os.getenv("LLM_BASE_URL"),
            api_key=os.getenv("LLM_API_KEY"),
            model=os.getenv("LLM_EMOTION_AGENT", "qwen2.5-32b-instruct"),
            verbose=verbose,
        )

    def _tools():
        def а(query: str):
            # Простая логика для классификации эмоций, можно расширить или улучшить
            emotions = {
                "гнев": ["злюсь", "ненавижу", "гнев", "злость"],
                "радость": ["рад", "счастлив", "весело", "счастье"],
                "нейтральное": ["нормально", "спокойно", "нейтрально"],
                "тоска": ["грустно", "тоска", "печаль", "сожаление"],
                "удивление": ["удивлен", "шокирован", "восторг"]
            }

            for emotion, keywords in emotions.items():
                if any(keyword in query.lower() for keyword in keywords):
                    return emotion
            return "Неизвестно"

# Тестируем
emotion_agent = EmotionAgent(verbose=True)
emotion = emotion_agent.invoke("Мне так грустно, я просто не знаю, что делать...")
print("Эмоция:", emotion)


AttributeError: type object 'EmotionAgent' has no attribute '_tools'

In [52]:
import os
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

class EmotionAgent:
    def __init__(self, verbose: bool = False):
        self.llm = ChatOpenAI(
            base_url=os.getenv("LLM_BASE_URL"),
            api_key=os.getenv("LLM_API_KEY"),
            model=os.getenv("LLM_EMOTION_AGENT", "qwen2.5-32b-instruct"),
            temperature=0,
            verbose=verbose,
        )

        self.prompt = ChatPromptTemplate.from_messages([
            ("system", "Ты — классификатор эмоций. Выбери одну из: гнев, радость, нейтральное, тоска, удивление. Отвечай только одним словом."),
            ("human", "{text}")
        ])

        self.chain = self.prompt | self.llm | StrOutputParser()

    def invoke(self, input_text: str) -> str:
        return self.chain.invoke({"text": input_text})


In [57]:
import os
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain.agents import AgentExecutor, AgentType, initialize_agent
from langchain_core.runnables import Runnable


class EmotionAgent(Runnable):
    def __new__(cls, verbose: bool = False):
        # Создаём LLM
        llm = ChatOpenAI(
            base_url=os.getenv("LLM_BASE_URL"),
            api_key=os.getenv("LLM_API_KEY"),
            model=os.getenv("LLM_EMOTION_AGENT", "qwen2.5-32b-instruct"),  # или qwen
            temperature=0,
            verbose=verbose,
        )

        # Создаём промпт
        prompt = ChatPromptTemplate.from_messages([
            ("system", "Ты — классификатор эмоций. Определи одну из: гнев, радость, нейтральное, тоска, удивление. Отвечай только одним словом."),
            ("human", "{text}")
        ])

        # Связываем всё в цепочку
        chain = prompt | llm | StrOutputParser()

        return chain


In [58]:
emotion_agent = EmotionAgent(verbose=True)
emotion = emotion_agent.invoke("Мне так грустно, я просто не знаю, что делать...")
print("Эмоция:", emotion)


Эмоция: тоска


In [68]:
class IntentAgent(BaseAgent):
    def __new__(cls, verbose: bool = False) -> AgentExecutor:
        llm = cls._llm(verbose)
        tools = cls._tools(llm)
        agent = initialize_agent(
            tools=tools,
            llm=llm,
            agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
            verbose=verbose,
        )
        return agent

    def _llm(verbose: bool):
        return ChatOpenAI(
            base_url=BaseAgent.LLM_BASE_URL,
            api_key=BaseAgent.LLM_API_KEY,
            model=os.getenv("LLM_INTENT_AGENT", "qwen2.5-32b-instruct"),
            temperature=0,
            verbose=verbose,
        )

    def _tools(llm):
        prompt = ChatPromptTemplate.from_messages([
            ("system", "Ты выбираешь наиболее подходящую тему для сообщения пользователя."),
            ("system", "Доступные темы: 'проблема с оплатой', 'жалоба', 'техническая проблема', 'вопрос по заказу', 'другое'."),
            ("system", "Отвечай только одной темой из списка, без объяснений."),
            ("human", "{text}")
        ])
        chain = prompt | llm | StrOutputParser()

        def classify_topic(query: str) -> str:
            return chain.invoke({"text": query})

        return [
            Tool(
                name="classify_topic",
                func=classify_topic,
                description="Выбирает наиболее подходящую тему обращения на русском языке.",
            ),
        ]


In [70]:
intent_agent = IntentAgent(verbose=True)
result = intent_agent.invoke("Я оплатил подписку, но сервис всё ещё недоступен")
print("Намерение:", result["output"])




> Entering new AgentExecutor chain...
Я должен определить, к какой категории относится этот вопрос, чтобы лучше понять, как на него ответить.
Action: classify_topic
Action Input: Я оплатил подписку, но сервис всё ещё недоступен
Observation: проблема с оплатой
Thought:I now know the final answer
Final Answer: Похоже, что у вас возникли проблемы с доступом к сервису после оплаты подписки. Рекомендую вам связаться с поддержкой сервиса, чтобы они могли проверить статус вашей оплаты и решить проблему доступа.

> Finished chain.
Намерение: Похоже, что у вас возникли проблемы с доступом к сервису после оплаты подписки. Рекомендую вам связаться с поддержкой сервиса, чтобы они могли проверить статус вашей оплаты и решить проблему доступа.
